####IMPORTAMOS LAS LIBRERIAS NECESARIAS####

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#este comando nos permite hacer visualizaciones en linea
%matplotlib inline

##Cargamos nuestro dataset Boston House Pricing y hacemos un reconocimiento del dataset##

In [ ]:
from sklearn.datasets import fetch_california_housing

In [ ]:
boston=fetch_california_housing()

In [ ]:
type(boston)

In [ ]:
# se utiliza para obtener una lista de las claves almacenadas en la variable o
# estructura de datos "boston"
boston.keys()

In [ ]:
#veamos que contiene la variable DESC (description) del dataset
print(boston.DESCR)

In [ ]:
#mostramos que contiene el dataset
print(boston.data)

In [ ]:
#analizamos los precios de las casas
print(boston.target)

In [ ]:
#observammos los nombres de las variables
print(boston.feature_names)

###Preparando el conjunto de datos###

In [ ]:
#creamos un dataframe con los datos de boston
#y pedimos ver los nombres de las columnas
dataset = pd.DataFrame(boston.data, columns=boston.feature_names)
dataset.head(5)

In [ ]:
#creo una nueva columna para mi variable dependiente
# que en este caso es lo que quiero predecir, la renombro Price en lugar de
# Target
dataset["Price"]=boston.target
dataset.head(5)

In [ ]:
#hacemos un pequeño análisis
dataset.info()

In [ ]:
#Resumimos las estadisticas del dataset
#solo nos servirá para las variables numéricas
dataset.describe( )

In [ ]:
#verificamos que no haya valores perdidos
#este paso es muy importante, .sum no sindica cuantoas valores nulos hay
dataset.isnull().sum()

In [ ]:
#EDA (Exploratory Data Analysis)
#como haremos una regresion lineal lo primero a analizar es la correlacion
dataset.corr()

In [ ]:
#creamos un diagrama de dispersion
import seaborn as sb
sb.pairplot(dataset)

## Conclusiones a partir de la matriz de correlación:

**Precio:**

* Tiene una correlación positiva con factores como MedInc (Ingreso Mediano), AveRooms (Habitaciones Promedio) y AveBedrms (Dormitorios Promedio). Esto significa que las casas en áreas con mayor ingreso mediano, más habitaciones y más dormitorios tienden a tener precios más altos.
* No tiene una correlación fuerte con Población o AveOccup (Ocupación Promedio). Esto sugiere que el precio no está necesariamente influenciado por la densidad de población general o el número promedio de personas que viven en una casa en un área.
* Tiene una débil correlación negativa con Latitud y Longitud. Es difícil hacer fuertes inferencias a partir de una débil correlación negativa, pero podría indicar una ligera preferencia de precio por casas en ciertas ubicaciones geográficas.

**MedInc (Ingreso Mediano):**

* Tiene una débil correlación negativa con HouseAge (Antigüedad de la Vivienda). Esto podría significar que podría haber una ligera tendencia a que las casas más nuevas estén en áreas con menor ingreso mediano.

**Antigüedad de la Vivienda:**

* No tiene una correlación fuerte con la mayoría de los demás factores.

**Habitaciones Promedio y Dormitorios Promedio:**

* Tienen una correlación positiva muy fuerte entre sí (como se esperaba). Esto significa que las casas con más habitaciones tienden a tener también más dormitorios.
* Tienen un patrón de correlación similar con Precio (correlación positiva) como se discutió anteriormente para Precio.

**Latitud y Longitud:**

* Tienen una correlación negativa muy fuerte entre sí. Esta correlación negativa perfecta indica que proporcionan la misma información, esencialmente representando la ubicación. Podría ser redundante usar ambos en un modelo.

**En general:**

La matriz de correlación sugiere que factores como el ingreso mediano, la cantidad de habitaciones y dormitorios son aspectos importantes que influyen en los precios de las viviendas. También sugiere que la ubicación (representada por Latitud y Longitud) podría ser un factor, pero usar solo uno de ellos podría ser suficiente. Sin embargo, es importante tener en cuenta que la correlación no implica causalidad. Se necesitaría un análisis más profundo para comprender las relaciones causales entre estos factores y el precio.


In [ ]:
#creamos un plot de dispersion
plt.scatter(dataset['MedInc'], dataset['Price'])
plt.xlabel("Ingreso Medio")
plt.ylabel("Precio")

In [ ]:
#correlacion entre el ingreso medio y el precio de la casa
sb.regplot(x="MedInc", y="Price", data= dataset)

##DEFINIMOS VARIABLES INDEPENDIENTES Y DEPENDIENTES##

In [ ]:
x=dataset.iloc[:,:-1]
y=dataset.iloc[:,-1]

In [ ]:
x.head(3)

In [ ]:
y.head(3)

##DIVIDIR EL DATASET EN DATOS DE ENTRENAMIENTO Y TESTEO##


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
#el 80% de los datos entrenaran el modelo
x_train

In [ ]:
#el 20% de los datos haran las pruebas del modelo
x_test

##estandarizamos el dataset##


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [ ]:
x_train = scaler.fit_transform(x_train)

In [ ]:
x_test = scaler.transform(x_test)

##MODEL TRAINING##

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
regression = LinearRegression()

In [ ]:
regression.fit(x_train, y_train)

In [ ]:
#vemos los coeficientes y los interceptos
print(regression.coef_)

In [ ]:
print(regression.intercept_)

In [ ]:
#sobre que parametros el modelo ha sido entrenado
regression.get_params()

##PREDICCION DEL MODELO CON DATOS DE PRUEBA##

In [ ]:
reg_pred = regression.predict(x_test)

In [ ]:
#vemos las predicciones del modelo
reg_pred

##SUPOSICIONES##

In [ ]:
#trazamos un diagrama de dispercion para las predicciones
plt.scatter(y_test, reg_pred)

In [ ]:
#errores residuales del modelo
residuals = y_test-reg_pred

In [ ]:
residuals

In [ ]:
# creamos grafico para los residuos y ver los errores
sb.displot(residuals, kind= "kde")

In [ ]:
#creamos grafica respecto a las predicciones y los residuos
#distribucion uniforme
plt.scatter(reg_pred, residuals)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

print(mean_absolute_error(y_test, reg_pred))
print(mean_squared_error(y_test, reg_pred))
print(np.sqrt(mean_squared_error(y_test, reg_pred)))

##APLICAMOS R CUADRADO Y R CUADRADO AJUSTADO (METRICAS DE RENDIMIENTO)##
FORMULAS:  
###R^2=1-SSR/SST###
DONDE R^2= COEFICIENTE DE DETERMINACION\
SSR= SUM OF SQUARES OF RESIDUALS\
SST= SUM OF SQUARES TOTAL\
###R^2 AJUSTADA=1-[(1-R2)^(n-1)/(n-k-1)]###
DONDE R^2= R^2 DEL MODELO\
n=NUMERO DE OBSERVACIONES\
k=NUMERO DE VARIABLES PREDICTORAS

In [ ]:
#r^2
from sklearn.metrics import r2_score
score=r2_score(y_test, reg_pred)
print(score)

In [ ]:
#^r^2 ajustado
1-(1-score)*(len(y_test)-1)/(len(y_test)-x_test.shape[1]-1)

##PREDECIR NUEVA DATA##

In [ ]:
boston.data[0].reshape(1, -1)

In [ ]:
#transformamos la nueva data
scaler.transform(boston.data[0].reshape(1, -1))

In [ ]:
regression.predict(scaler.transform(boston.data[0].reshape(1, -1)))

##Serializar el archivo del modelo para su implementación##

In [ ]:
import pickle

In [ ]:
pickle.dump(regression, open('regmodel.pkl', 'wb'))

In [ ]:
pickled_model=pickle.load(open('regmodel.pkl', 'rb'))

In [ ]:
pickled_model.predict(scaler.transform(boston.data[0].reshape(1, -1)))